<a href="https://colab.research.google.com/github/Gau-6426/PCAP/blob/main/Code/segment_status.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [11]:
df_segments_status = pd.read_csv("https://raw.githubusercontent.com/Gau-6426/PCAP/main/Data/segment_status.csv")
print(df_segments_status.head(10))
df_segments_status.shape

   _id                updated_at  segment_id  velocity
0    0  2020-07-03T14:55:31.869Z       24845        20
1    1  2020-07-03T15:02:56.048Z       33923        10
2    2  2020-07-04T08:15:52.696Z       33824         5
3    3  2020-07-04T08:15:59.903Z       33824         5
4    4  2020-07-04T08:16:08.201Z       33824         5
5    5  2020-07-04T08:16:11.070Z       33824         5
6    6  2020-07-04T08:16:18.512Z       33824         5
7    7  2020-07-04T08:16:21.798Z       33824         5
8    8  2020-07-04T08:16:24.746Z       33824         5
9    9  2020-07-04T08:16:27.606Z       33824         5


(90938, 4)

In [6]:
segment_status_sort=df_segments_status.sort_values(by='velocity')
segment_status_sort

,_id,updated_at,segment_id,velocity
32416,32416,2020-08-03T00:20:31.797Z,29164,0
12828,12828,2020-08-03T00:05:30.888Z,83584,0
8119,8119,2020-08-03T00:02:28.071Z,6536,0
7466,7466,2020-08-02T23:59:32.926Z,62128,0
33760,33760,2020-08-03T00:23:27.901Z,5562,0
...,...,...,...,...
50856,50856,2020-12-02T15:33:41.160Z,14409,199
42002,42002,2020-11-26T01:00:22.535Z,63453,199
52438,52438,2020-12-04T02:19:19.741Z,68714,200
49320,49320,2020-12-02T02:21:46.673Z,10099,200


In [24]:
Segments_Status = pd.read_csv("https://raw.githubusercontent.com/Gau-6426/PCAP/main/Data/segment_status.csv", index_col="_id", parse_dates=["updated_at"])

In [32]:
from math import ceil

def transform_LOS(segment_id, velocity):
    velocity = df_segments_status.loc[segment_id, "velocity"]
    if velocity is None:
        velocity = 50
    
    # Transform to label
    labels = ["A", "B", "C", "D", "E", "F"]
    threshold = 35
    if velocity >= 70:
        threshold = 45
    elif velocity >= 60:
        threshold = 40

    t = max(threshold - velocity, 0)
    return labels[min(ceil(t / 5), 5)]

def transform_report(row):
    """
    @Params:
        dt: Timestamp object of Pandas
    @Return:
        dict: {"date", "period_{hour}_{00|30}"}
    """
    LOS = transform_LOS(row["segment_id"], row["velocity"])
    dt = row["updated_at"]
    intervals = list(range(24))
    h = dt.hour
    m = "00" if dt.minute < 30 else "30"
    p_name = f"period_{h}_{m}"
    return dt.date(), dt.weekday(), p_name, LOS

In [33]:
def major_voting(labels):
    unique_labels = set(labels)
    count_labels = [labels.count(label) for label in unique_labels]

    sorted_labels = sorted(zip(unique_labels, count_labels), key=lambda x: x[1])
    if len(sorted_labels) > 1 and sorted_labels[0][1] == sorted_labels[1][1]:
        print("Oh no, many majors?")
    return sorted_labels[0][0]

def mean_voting(labels):
    l = ["A", "B", "C", "D", "E", "F"]
    values = {"A":0, "B":1, "C":2, "D":3, "E":4, "F":5}
    mean = sum(values[label] for label in labels) / len(labels)
    return l[min(round(mean), 5)]

In [35]:
compress_LOS = df_segments_status.groupby(by=["segment_id", "date", "weekday", "period"])["LOS"].apply(list)
compress_LOS = pd.DataFrame(compress_LOS).reset_index()
compress_LOS["LOS"] = compress_LOS["LOS"].apply(mean_voting)
compress_LOS

KeyError: ignored